## **Limpieza del CSV de "Humedad del aire a 2 metros"**
----

#### ***Importación de paquetes***

In [7]:
# make sure to install these packages before running:
# pip install pandas
# pip install sodapy

import pandas as pd
import ssl

ssl._create_default_https_context = ssl._create_unverified_context

#### ***Cargar el CSV mediante Pandas***

In [8]:
df = pd.read_csv("../CSV/humedad-aire.csv")
df

URLError: <urlopen error [SSL: CERTIFICATE_VERIFY_FAILED] certificate verify failed: unable to get local issuer certificate (_ssl.c:1000)>

#### ***Enfocarnos en los departamentos de la región Pacífica***

In [ ]:
#df.head()
df["departamento"].value_counts()

#### ***Liberarnos de ciertas columnas***

In [ ]:
df = df.drop(["codigosensor", "codigoestacion", "descripcionsensor"], axis=1)
df.head()

#### ***Aplicar upcase en los títulos de las columnas***

In [ ]:
df.columns = [col.upper() for col in df.columns]
df.columns.tolist()

#### ***Guardar la nueva versión del dataset***

In [ ]:
df.to_csv("../HumedadAire_RegionPacifico.csv", index=False)